In [1]:
import os
from pathlib import Path
import shutil
import pandas as pd

# Пути к исходным директориям
evklid_dir = Path("D:/dataset/_eyepacs/data/center_evklid/train")
cosine_dir = Path("D:/dataset/_eyepacs/data/filtered_data_with_cosine/train")

# Путь к финальной директории и CSV
combined_dir = Path("D:/dataset/_eyepacs/data/combined_cosine_evklid")
combined_dir.mkdir(parents=True, exist_ok=True)
csv_output_path = Path("D:/dataset/_eyepacs/data/combined_cosine_evklid.csv")

# Собираем имена файлов из обеих папок
def get_all_images_by_label(root_dir):
    image_dict = {}
    for label in ['0', '1', '2']:
        folder = root_dir / label
        if folder.exists():
            image_dict[label] = {p.name for p in folder.glob("*.jpg")}
    return image_dict

evklid_images = get_all_images_by_label(evklid_dir)
cosine_images = get_all_images_by_label(cosine_dir)

# Подготовим данные для csv
csv_rows = []

# Пересекаем и копируем
for label in ['0', '1', '2']:
    common_images = evklid_images.get(label, set()) & cosine_images.get(label, set())
    print(f"Класс {label}: найдено {len(common_images)} общих изображений")

    # Создаем директорию
    dst_label_dir = combined_dir / label
    dst_label_dir.mkdir(parents=True, exist_ok=True)

    for img_name in common_images:
        src_path = evklid_dir / label / img_name  # Можно поменять на cosine_dir
        dst_path = dst_label_dir / img_name
        shutil.copy2(src_path, dst_path)

        # Добавим строку в csv
        csv_rows.append({"image_name": img_name, "true_label": int(label)})

# Сохраняем CSV
df = pd.DataFrame(csv_rows)
df.to_csv(csv_output_path, index=False)
print(f"CSV сохранён по пути: {csv_output_path}")


Класс 0: найдено 18867 общих изображений
Класс 1: найдено 5708 общих изображений
Класс 2: найдено 86 общих изображений
CSV сохранён по пути: D:\dataset\_eyepacs\data\combined_cosine_evklid.csv
